In [122]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import Select

import pandas as pd
from bs4 import BeautifulSoup
import requests

class Navegador:
    def __init__(self):
        # Configurar opções do Chrome
        options = Options()
        options.add_argument("--enable-automation")
        options.add_argument("--start-maximized")
        options.add_argument("--disable-notifications")
        options.add_argument("--disable-popup-blocking")
        options.add_argument("--kiosk-printing")


        # Inicializar o WebDriver do Chrome com as opções configuradas
        #self.driver = webdriver.Remote(command_executor="http://localhost:4444/wd/hub", options=options)
        self.driver = webdriver.Chrome(options=options)
        self.wait = WebDriverWait(self.driver, 10)
        self.by = By
        self.locator = {
            "XPATH": By.XPATH,
            "ID": By.ID,
            "CLASS_NAME": By.CLASS_NAME,
            "LINK_TEXT": By.LINK_TEXT,
            "NAME": By.NAME,
            "PARTIAL_LINK_TEXT": By.PARTIAL_LINK_TEXT,
            "TAG_NAME": By.TAG_NAME
        }

    def get_session_id (self):
        return self.driver.session_id

    def disable_alert(self):
        self.driver.switch_to.alert.dismiss()

    def element_get_text(self, element, tag):
        if element in self.locator:
            try:
                # Aguardar até que o elemento seja visível e, em seguida, retornar seu texto
                element_text = self.wait.until(EC.visibility_of_element_located((self.locator[element], tag)))
                return element_text
            except TimeoutException:
                print("Elemento não encontrado")   
                  
    def get_elements(self, element, tag):
        if element in self.locator:
            try:
                # Aguardar até que o elemento seja visível e, em seguida, retornar seu texto
                elements = self.wait.until(EC.visibility_of_all_elements_located((self.locator[element], tag)))
                return elements
            except TimeoutException:
                print("Elemento não encontrado")

    def get(self, url):
        # await asyncio.sleep(0)
        self.driver.get(url)
    def close(self):
    #  await asyncio.sleep(0)
        self.driver.quit()   

    def close_session(self, session_id):
        grid_url = "https://grid.consium.com.br/wd/hub"
        session_url = f"{grid_url}/session/{session_id}"
        response = requests.delete(session_url)
        if response.status_code == 200:
            print("Sessão fechada com sucesso!")
        else:
            print("Falha ao fechar a sessão.")

        return response    
    # Funcao para digitar no elemento           
    def sendkeys(self, element, tag, keys):
    #  await asyncio.sleep(0)
        if element in self.locator:
            try:
                self.wait.until(EC.presence_of_element_located((self.locator[element], tag))).send_keys(keys)
            except TimeoutException:
                print("Elemento não encontrado")
                
    # Funcao para clicar no elemento                
    def click(self, element, tag):
    #  await asyncio.sleep(0)
        if element in self.locator:
            try:
                self.wait.until(EC.visibility_of_element_located((self.locator[element], tag))).click()
            except TimeoutException:    
                print("Elemento não encontrado")


    def get_table_element(self, element, tag):
        try:
            # Obter o conteúdo HTML da tag <tbody>
            html_content = self.wait.until(EC.visibility_of_element_located((self.locator[element], tag))).get_attribute('innerHTML')
            # Extrair dados da tabela e transforma em dataframe
            data = self.table_to_dataframe(html_content)
            qtd_linhas = len(data)
            return data, qtd_linhas
        except TimeoutException:
            print("Elemento não encontrado")

    def table_to_dataframe(self, html_content):

        soup = BeautifulSoup(html_content, 'html.parser')

        # Encontra a tabela desejada (selecionando-a pela classe, id ou outras características)
        table = soup.find('table')

        # Verifica se a tabela foi encontrada
        if table:
            # Inicializa uma lista para armazenar os dados da tabela
            table_data = []
            # Itera sobre as linhas da tabela (<tr>)
            for row in table.find_all('tr'):
                # Inicializa uma lista para armazenar os dados de uma linha
                row_data = []
                # Itera sobre as células da linha (<td>)
                for cell in row.find_all(['td']):
                    # Adiciona o texto da célula à lista de dados da linha
                    value = cell.text.strip()
                    # Verifica se o valor não está vazio
                    if value:
                        row_data.append(value)
                    else:
                        row_data.append(None)
                    # Verifica se a célula contém uma tag de âncora (hiperlink)
                    link = cell.find('a')
                    if link:
                        # Se houver uma tag de âncora, adiciona o link (href) à lista de dados da linha
                        row_data.append(link.get('href'))
                    else:
                        row_data.append(None)
                # Adiciona os dados da linha à lista de dados da tabela
                if row_data:
                    table_data.append(row_data)

            # Imprime os dados da tabela
            
            df = pd.DataFrame(table_data)
            df.to_excel('arquivo.xlsx', index=False)

            return df 
        

                   

In [242]:
import sqlite3

class Sqclass:
    def __init__(self):
        self.conn = sqlite3.connect('../database/database.db')
        self.cursor = self.conn.cursor()
        self.create_table_lists()
        self.create_table_products()
        self.create_table_products_infos()
        
    def close(self):
        self.conn.close()
    
    def create_table_lists(self):
        self.cursor.execute(""" 
                            CREATE TABLE IF NOT EXISTS product_list (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                name TEXT,
                                url TEXT
                            )
                            """)
        self.conn.commit()
    
    def create_table_products(self):
        self.cursor.execute(""" 
                            CREATE TABLE IF NOT EXISTS products (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                title TEXT,
                                photo_src TEXT,
                                product_site_id TEXT,
                                id_list INTEGER,
                                FOREIGN KEY(id_list) REFERENCES product_list(id)
                            )
                            """)
        self.conn.commit()
    
    def create_table_products_infos(self):
        self.cursor.execute(""" 
                            CREATE TABLE IF NOT EXISTS products_infos (
                                id INTEGER PRIMARY KEY AUTOINCREMENT,
                                site_option_id TEXT,
                                size TEXT,
                                price_web REAL,
                                price_b2b REAL,
                                id_product INTEGER,
                                FOREIGN KEY(id_product) REFERENCES products(id)
                            )
                            """)
        self.conn.commit()

    
    def insert_into_product_list(self, name, url):
        self.cursor.execute("""
                            INSERT INTO product_list (name, url)
                            VALUES (?, ?)
                            """, (name, url))
        self.conn.commit()
        return self.cursor.lastrowid  # Retorna o ID do registro inserido
    
    
    def get_product_list(self):
        self.cursor.execute("SELECT * FROM product_list")
        return self.cursor.fetchall()
    
    # Método para inserir dados na tabela products verifica se o produto ja existe comparando o product_site_id caso exista faz o update
# Método para inserir ou atualizar dados na tabela 'products'
    def insert_or_update_product(self, title, photo_src, product_site_id, id_list):
        # Verificar se o produto já existe
        self.cursor.execute("SELECT id FROM products WHERE product_site_id = ?", (product_site_id,))
        existing_product = self.cursor.fetchone()
        
        if existing_product:
            product_id = existing_product[0]
            # Se o produto existir, faça a atualização
            self.cursor.execute("""
                UPDATE products
                SET title = ?, photo_src = ?, id_list = ?
                WHERE product_site_id = ?
            """, (title, photo_src, id_list, product_site_id))
        else:
            # Se o produto não existir, faça a inserção
            self.cursor.execute("""
                INSERT INTO products (title, photo_src, product_site_id, id_list)
                VALUES (?, ?, ?, ?)
            """, (title, photo_src, product_site_id, id_list))
            product_id = self.cursor.lastrowid
        
        # Confirma a transação no banco de dados
        self.conn.commit()
        
        # Retorna o ID do produto existente ou recém-inserido
        return product_id


    
    # Método para inserir dados na tabela products_infos verifica se as informações ja existe comparando o site_option_id caso exista faz o update
    def insert_or_update_products_infos(self, site_option_id, size, price_web, price_b2b, id_product):
        # Verificar se as informações do produto já existem
        self.cursor.execute("SELECT id FROM products_infos WHERE site_option_id = ?", (site_option_id,))
        existing_info = self.cursor.fetchone()
        
        if existing_info:
            info_id = existing_info[0]
            # Se as informações existirem, faça a atualização
            self.cursor.execute("""
                UPDATE products_infos
                SET size = ?, price_web = ?, price_b2b = ?, id_product = ?
                WHERE site_option_id = ?
            """, (size, price_web, price_b2b, id_product, site_option_id))
        else:
            # Se as informações não existirem, faça a inserção
            self.cursor.execute("""
                INSERT INTO products_infos (site_option_id, size, price_web, price_b2b, id_product)
                VALUES (?, ?, ?, ?, ?)
            """, (site_option_id, size, price_web, price_b2b, id_product))
            info_id = self.cursor.lastrowid
        
        # Confirma a transação no banco de dados
        self.conn.commit()
        
        # Retorna o ID do registro existente ou recém-inserido
        return info_id



In [13]:
#VARIAVEIS
LOGIN_URL = 'https://b2b.tradeinn.com'
USER_EMAIL = 'vendas@oldfirm.com.br'
USER_PASSWORD = 'Kohlrauschrs18!G'

In [265]:
bot = Navegador()

bot.get(LOGIN_URL)

bot.sendkeys('ID', 'email_login', USER_EMAIL)
bot.sendkeys('ID', 'pass_login', USER_PASSWORD)

bot.click('XPATH', '/html/body/div[3]/div/div/div[2]/div[1]/form/div[4]/button')

In [274]:
import time
import sys

def login_simulation():
    print("Iniciando login...")
    steps = 10
    for i in range(steps):
        time.sleep(0.5)  # Simulando tempo de processamento
        sys.stdout.write(f"\rLogando: [{'#' * (i + 1)}{'.' * (steps - i - 1)}] {int((i + 1) / steps * 100)}%")
        sys.stdout.flush()

    print("\nLogin concluído com sucesso!")

login_simulation()


Iniciando login...
Logando: [##########] 100%
Login concluído com sucesso!


In [262]:
def get_products_site(LIST_ID, url):
    
    bot.get(url)
    import time
    
    elements = bot.get_elements('CLASS_NAME', 'product-listing-wrapper')

    i = 1
    refresh = 0

    start_time = time.time()  # Início da medição de tempo


    for element in elements:
        
        xpath = f"/html/body/div[3]/div[2]/div[1]/div/main/ul/li[{i}]"
        
        bot.click('XPATH', xpath)
        
        
        title = bot.element_get_text('ID', 'name_product')
        title = title.text

        photo_src = bot.element_get_text('XPATH', '/html/body/div[3]/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/img')
        photo_src = photo_src.get_attribute('src')
        
        product_site_id = bot.element_get_text('XPATH', xpath).get_attribute('data-id_modelo')
        
        #salvar no bd mas verificar se existe antes
        
        new_product = bd.insert_or_update_product(title, photo_src, product_site_id, LIST_ID)
        
        #salva listas
        
        drp_element = bot.get_elements('ID', 'tallas_productos')[0]
    # Criar uma instância de Select para interagir com o dropdown
        select = Select(drp_element)

        # Obter todas as opções
        options = select.options

        # Contar o número de opções
        num_options = len(options)

        for index in range(num_options):

            select.select_by_index(index)

            size = select.first_selected_option.text
            option_site_id = select.first_selected_option.get_attribute('value')
            price_web = bot.element_get_text('ID', 'precio_web').text
            price_b2b = bot.element_get_text('ID', 'precio_b2b').text
            
            new_product_info = bd.insert_or_update_products_infos(option_site_id, size, price_web, price_b2b, new_product)

        bot.click('ID', 'js-cerrar-detalle')
        i = i + 1
        print(i)
        print(f'REFRESH TEMP: {refresh}')
        refresh = refresh + 1
        
        if refresh == 8:
            refresh = 0
            bot.get(url)

    end_time = time.time()  # Fim da medição de tempo
    execution_time = end_time - start_time  # Cálculo do tempo de execução 

    print(execution_time)

In [264]:
def update_all_products():
    import time

    bd = Sqclass()

    listas = bd.get_product_list()

    elements = bot.get_elements('CLASS_NAME', 'product-listing-wrapper')

    for list in listas:
        
        url = list[2]
        LIST_ID = list[0]
        
        get_products_site(LIST_ID, url)
    

Elemento não encontrado


KeyboardInterrupt: 

In [250]:
elements = bot.get_elements('CLASS_NAME', 'product-listing-wrapper')

i = 1
refresh = 0

start_time = time.time()  # Início da medição de tempo


for element in elements:
    
    xpath = f"/html/body/div[3]/div[2]/div[1]/div/main/ul/li[{i}]"
    
    bot.click('XPATH', xpath)
    
    
    title = bot.element_get_text('ID', 'name_product')
    title = title.text

    photo_src = bot.element_get_text('XPATH', '/html/body/div[3]/div[1]/div/div[2]/div[1]/div[2]/div/div[1]/img')
    photo_src = photo_src.get_attribute('src')
    
    product_site_id = bot.element_get_text('XPATH', xpath).get_attribute('data-id_modelo')
    
    #salvar no bd mas verificar se existe antes
    
    new_product = bd.insert_or_update_product(title, photo_src, product_site_id, LIST_ID)
    
    #salva listas
    
    drp_element = bot.get_elements('ID', 'tallas_productos')[0]
# Criar uma instância de Select para interagir com o dropdown
    select = Select(drp_element)

    # Obter todas as opções
    options = select.options

    # Contar o número de opções
    num_options = len(options)

    for index in range(num_options):

        select.select_by_index(index)

        size = select.first_selected_option.text
        option_site_id = select.first_selected_option.get_attribute('value')
        price_web = bot.element_get_text('ID', 'precio_web').text
        price_b2b = bot.element_get_text('ID', 'precio_b2b').text
        
        new_product_info = bd.insert_or_update_products_infos(option_site_id, size, price_web, price_b2b, new_product)

    bot.click('ID', 'js-cerrar-detalle')
    i = i + 1
    print(i)
    print(f'REFRESH TEMP: {refresh}')
    refresh = refresh + 1
    
    if refresh == 8:
        refresh = 0
        bot.get(url)

end_time = time.time()  # Fim da medição de tempo
execution_time = end_time - start_time  # Cálculo do tempo de execução 

print(execution_time)

2
REFRESH TEMP: 0
3
REFRESH TEMP: 1
4
REFRESH TEMP: 2
5
REFRESH TEMP: 3
6
REFRESH TEMP: 4
7
REFRESH TEMP: 5
8
REFRESH TEMP: 6
9
REFRESH TEMP: 7
10
REFRESH TEMP: 8
11
REFRESH TEMP: 9
12
REFRESH TEMP: 0
13
REFRESH TEMP: 1
14
REFRESH TEMP: 2
15
REFRESH TEMP: 3
16
REFRESH TEMP: 4
17
REFRESH TEMP: 5
18
REFRESH TEMP: 6
19
REFRESH TEMP: 7
20
REFRESH TEMP: 8
21
REFRESH TEMP: 9
22
REFRESH TEMP: 0
23
REFRESH TEMP: 1
24
REFRESH TEMP: 2
25
REFRESH TEMP: 3
26
REFRESH TEMP: 4
27
REFRESH TEMP: 5
28
REFRESH TEMP: 6
29
REFRESH TEMP: 7
30
REFRESH TEMP: 8
31
REFRESH TEMP: 9
32
REFRESH TEMP: 0
33
REFRESH TEMP: 1
34
REFRESH TEMP: 2
35
REFRESH TEMP: 3
36
REFRESH TEMP: 4
37
REFRESH TEMP: 5
38
REFRESH TEMP: 6
39
REFRESH TEMP: 7
40
REFRESH TEMP: 8
41
REFRESH TEMP: 9
42
REFRESH TEMP: 0
43
REFRESH TEMP: 1
44
REFRESH TEMP: 2
45
REFRESH TEMP: 3
46
REFRESH TEMP: 4
47
REFRESH TEMP: 5
48
REFRESH TEMP: 6
49
REFRESH TEMP: 7
50
REFRESH TEMP: 8
51
REFRESH TEMP: 9
52
REFRESH TEMP: 0
53
REFRESH TEMP: 1
54
REFRESH TEMP: 2
5

In [140]:

bot.click('ID', 'tallas_productos')

drp_xpath = f'/html/body/div[3]/div[1]/div/div[2]/div[2]/form/div[2]/select/option[{a}]'

bot.click('XPATH', drp_xpath)
time.sleep(2)

Elemento não encontrado


TypeError: 'NoneType' object is not subscriptable

In [45]:
size
price_web
price_b2b

'adidas Mundial Team football boots\n£ 80.80\nView product'

In [46]:
bot.click('PARTIAL_LINK_TEXT', elements[0].text)

Elemento não encontrado
